Original dataset found [here](https://www.kaggle.com/akulbahl/covid19-airline-flight-delays-and-cancellations)

## Import of cuML

In [ ]:
!nvidia-smi

Tue Nov 24 22:28:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 185 (delta 5), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (185/185), 57.14 KiB | 11.43 MiB/s, done.
Resolving deltas: 100% (67/67), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT CHANGES: RAPIDS on Colab will be pegged to 0.14 Stable until further notice.
2. Default stable version is now 0.14.  Nightly will redirect to 0.14.
3. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.14, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rap

In [ ]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
!pip install tpot
from tpot import TPOTClassifier
drive.mount('/content/drive')

path_to_dataset = '/content/drive/My Drive/Coding/jantojun2020.csv'
dataset = pd.read_csv(path_to_dataset)

     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 163kB 17.9MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=02c51a8ee34f49b4f16188143d9e760d9ee3bce181e2a6ffa6a44c2442a905b8
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dataset.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2020,1,1,1,3,1/1/2020,WN,5888,N951WN,ONT,"Ontario, CA",CA,California,SFO,"San Francisco, CA",CA,California,1810,1851.0,41.0,41.0,1.0,2.0,1800-1859,44.0,1935.0,2049.0,4.0,1945,2053.0,68.0,68.0,1.0,4.0,1900-1959,0,NaN,95.0,122.0,74.0,363,2,8.0,0.0,27.0,0.0,33.0
1,2020,1,1,1,3,1/1/2020,WN,6276,N467WN,ONT,"Ontario, CA",CA,California,SFO,"San Francisco, CA",CA,California,1150,1146.0,-4.0,0.0,0.0,-1.0,1100-1159,13.0,1159.0,1310.0,8.0,1320,1318.0,-2.0,0.0,0.0,-1.0,1300-1359,0,NaN,90.0,92.0,71.0,363,2,NaN,NaN,NaN,NaN,NaN
2,2020,1,1,1,3,1/1/2020,WN,4598,N7885A,ONT,"Ontario, CA",CA,California,SJC,"San Jose, CA",CA,California,2020,2016.0,-4.0,0.0,0.0,-1.0,2000-2059,8.0,2024.0,2121.0,3.0,2130,2124.0,-6.0,0.0,0.0,-1.0,2100-2159,0,NaN,70.0,68.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN
3,2020,1,1,1,3,1/1/2020,WN,4761,N551WN,ONT,"Ontario, CA",CA,California,SJC,"San Jose, CA",CA,California,1340,1350.0,10.0,10.0,0.0,0.0,1300-1359,10.0,1400.0,1503.0,2.0,1455,1505.0,10.0,10.0,0.0,0.0,1400-1459,0,NaN,75.0,75.0,63.0,333,2,NaN,NaN,NaN,NaN,NaN
4,2020,1,1,1,3,1/1/2020,WN,5162,N968WN,ONT,"Ontario, CA",CA,California,SJC,"San Jose, CA",CA,California,915,916.0,1.0,1.0,0.0,0.0,0900-0959,6.0,922.0,1019.0,4.0,1035,1023.0,-12.0,0.0,0.0,-1.0,1000-1059,0,NaN,80.0,67.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN


## Feature processing

In [ ]:
dataset = dataset.drop(columns = ['YEAR', 'FL_DATE', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_NM', 'ORIGIN_STATE_ABR', 'DEST_CITY_NAME', 
                                  'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEP_DELAY_NEW', 'DEP_TIME_BLK','TAXI_OUT', 'WHEELS_OFF', 
                                  'WHEELS_ON', 'TAXI_IN', 'ARR_DELAY_NEW', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 
                                  'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'CANCELLATION_CODE', 'ARR_TIME_BLK', 'AIR_TIME'
                                   ])
dataset = dataset[dataset['TAIL_NUM'].notna()]

In [ ]:
def underSampling():
  dataset_cancel0 = dataset[dataset['CANCELLED']==0]
  dataset_cancel1 = dataset[dataset['CANCELLED']==1]
  drop_indices = np.random.choice(dataset_cancel0.index, 2342918, replace=False)
  dataset_cancel0 = dataset_cancel0.drop(drop_indices)
  return dataset_cancel0, dataset_cancel1

In [ ]:
def fillna(dataset_cancel0, dataset_cancel1):
  dataset_cancel1[['DEP_DELAY','DEP_DEL15','DEP_DELAY_GROUP','ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP']] = \
  dataset_cancel1[['DEP_DELAY','DEP_DEL15','DEP_DELAY_GROUP','ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP']].fillna(value=0)
  dataset_cancel1[['CRS_DEP_TIME', 'DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME']] = \
  dataset_cancel1[['CRS_DEP_TIME', 'DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME']].fillna(method='ffill', axis = 1)
  dataset_cancel0[['ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP']] = \
  dataset_cancel0[['ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP']].fillna(value=0)
  dataset_cancel0[['CRS_ARR_TIME', 'ARR_TIME', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME']] = \
  dataset_cancel0[['CRS_ARR_TIME', 'ARR_TIME', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME']].fillna(method='ffill', axis = 1)
  return dataset_cancel0, dataset_cancel1

In [ ]:
def data_encoding(dataset_cancel0, dataset_cancel1):
  dataset_appended = dataset_cancel0.append(dataset_cancel1)
  dummy_df = pd.get_dummies(dataset_appended[['MKT_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'TAIL_NUM']])
  dataset_appended = pd.concat([dataset_appended.drop(columns=['MKT_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'TAIL_NUM']),dummy_df], axis = 1)
  print(dataset_appended.shape)
  
  return dataset_appended

In [ ]:
def data_suffle(dataset_appended):
  dataset_appended = dataset_appended.sample(frac=1)
  dataset_appended = dataset_appended.sample(frac=0.05)
  dataset_y = dataset_appended["CANCELLED"]
  #dataset_y.to_csv("C:\Coding\Project\CISC889\Final project\input\dataset_y_suffle.csv", index = False)
  dataset_x = dataset_appended.drop(columns = ["CANCELLED"])
  #dataset_x.to_csv("C:\Coding\Project\CISC889\Final project\input\dataset_x_suffle.csv", index = False)
  return dataset_x, dataset_y

In [ ]:
dataset_cancel0, dataset_cancel1 = underSampling()
dataset_cancel0, dataset_cancel1 = fillna(dataset_cancel0, dataset_cancel1)
dataset_appended = data_encoding(dataset_cancel0, dataset_cancel1)
dataset_x, dataset_y = data_suffle(dataset_appended)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


(240002, 6702)


In [ ]:
dataset_x = dataset_x.to_numpy()
dataset_y = dataset_y.to_numpy().ravel()
np.info(dataset_y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9000, 6701)
(3000, 6701)
(9000,)
(3000,)


## TPOT Init with GPU

In [ ]:
tpot = TPOTClassifier(generations=5, population_size=40, verbosity=3, config_dict = "TPOT cuML", random_state=42, warm_start = True , max_time_mins = 60)

In [ ]:
tpot.fit(X_train, y_train)

19 operators have been imported by TPOT.


Skipped pipeline #22 due to time out. Continuing to the next pipeline.
Skipped pipeline #26 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'LogisticRegression(penalty='l1', tol=0.0001, C=0.1, fit_intercept=True, max_iter=1000, linesearch_max_iter=50, verbose=2, l1_ratio=None, solver='qn', handle=<cuml.common.handle.Handle object at 0x7f7a103c1030>)' (type <class 'cuml.linear_model.logistic_regression.LogisticRegression'>) doesn't.
_pre_test decorator: _random_mutation_operator: num_test=0 All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'KNeighborsClassifier(weights='uniform')' (type <class 'cuml.neighbors.kneighbors_classifier.KNeighborsClassifier'>) doesn't.
_pre_test decorator: _random_mutation_operator: num_test=0 All intermediate steps should be transfor

## Below is the code for using date features only

In [ ]:
dataset = dataset[['QUARTER', 'MONTH', 'DAY_OF_MONTH','CANCELLED']]

In [ ]:
def underSampling():
  dataset_cancel0 = dataset[dataset['CANCELLED']==0]
  dataset_cancel1 = dataset[dataset['CANCELLED']==1]
  drop_indices = np.random.choice(dataset_cancel0.index, 2179995, replace=False)
  dataset_cancel0 = dataset_cancel0.drop(drop_indices)
  dataset_cancel0 = dataset_cancel0.append(dataset_cancel1)
  return dataset_cancel0

In [ ]:
def data_suffle(dataset_appended):
  dataset_appended = dataset_appended.sample(frac=1)
  #dataset_appended = dataset_appended.sample(frac=0.05)
  dataset_y = dataset_appended["CANCELLED"]
  #dataset_y.to_csv("C:\Coding\Project\CISC889\Final project\input\dataset_y_suffle.csv", index = False)
  dataset_x = dataset_appended.drop(columns = ["CANCELLED"])
  #dataset_x.to_csv("C:\Coding\Project\CISC889\Final project\input\dataset_x_suffle.csv", index = False)
  return dataset_x, dataset_y

In [ ]:
dataset_appended = underSampling()
dataset_x, dataset_y = data_suffle(dataset_appended)

In [ ]:
dataset_x = dataset_x.to_numpy()
dataset_y = dataset_y.to_numpy().ravel()
np.info(dataset_y)
X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
tpot = TPOTClassifier(generations=5, population_size=40, verbosity=3, config_dict = "TPOT cuML", random_state=42, warm_start = True , max_time_mins = 60)

In [ ]:
tpot.fit(X_train, y_train)

19 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'KNeighborsClassifier(weights='uniform')' (type <class 'cuml.neighbors.kneighbors_classifier.KNeighborsClassifier'>) doesn't.
_pre_test decorator: _random_mutation_operator: num_test=0 All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'KNeighborsClassifier(weights='uniform')' (type <class 'cuml.neighbors.kneighbors_classifier.KNeighborsClassifier'>) doesn't.

Generation 1 - Current Pareto front scores:

-1	0.8690046205869045	XGBClassifier(input_matrix, XGBClassifier__alpha=10, XGBClassifier__learning_rate=1.0, XGBClassifier__max_depth=6, XGBClassifier__min_child_weight=2, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.3, XGBClassifier__tree_method=gpu_hist)
_pre_test decorator: _random_mutation_operator: num_test=0 All inter

TPOTClassifier(config_dict='TPOT cuML', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=60,
               memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=40,
               random_state=42, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=True)

In [ ]:
print(tpot.score(X_test, y_test))

0.8692732375250065


In [ ]:
tpot.export('/content/drive/My Drive/Coding/tpot_airflight_pipeline_date_only.py')